### Python and Just-in-Time Compilation:

This is a quick lecture to take a look at the ways to improve the performance of interpreted code. We have been working with compiled code with OpenMP in C, so now let's jump back to interpreted code in Python.

* _Compilation_ is the process of generating machine code that can be run directly on a processor from source code written in a programming language.

* _Interpreters_ are programs that directly execute instructions written in a programming language without compilation.

There is a lot of gray area and confusion and reality will often blend these concepts. Let's consider the Java landscape. Java source code is _compiled_ into a intermediate representation (Java byte code) that run through some combination of:
  * a Java _interpreter_ that translates Java byte code to machine instructions
  * a just-in-time (JIT) compiler that translates byte code into machine code as the program executes
  
Java ends up being a compiled language from a programmer's perspective. The interpreter provides portability.

The concept of just-in-time compilation on interpreted languages is powerful in that it can realize the performance of compiled code while preserving the flexibility and ease of programming in interpreted code. Python has a JIT and we will explore performance of JIT compilation and its interaction with parallelism.

#### Mutual Web Outlinks Example

Returning to our Web Outlinks example. Let's start with a review of 
  * building a small graph
  * building a larger graph 
  * the serial mutual outlinks program
  * a `joblib` parallelized outlinks program (based on processes)
  

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

G = nx.erdos_renyi_graph(5,0.8, directed=True)

In [ ]:
nx.draw_networkx(G, pos=nx.spring_layout(G), with_labels=False, node_size=50)

In [ ]:
gmat = nx.to_numpy_matrix(G)

outmat = np.zeros(gmat.shape, dtype=np.float32)

for i in range(gmat.shape[0]):
    for j in range(i+1,gmat.shape[1]):        
        outmat[i,j] = np.dot(np.asarray(gmat[i,:]).reshape(-1), np.asarray(gmat[j,:]).reshape(-1))
outmat

Now generate a big graph and we'll start our performance analysis. Let's look at a serial version on a graph with 1000 nodes and about 5 outbound edges per node. 

Serial program to calculate mutual outlinks.

In [ ]:
# cycles disappear when p = log(n)/n
G= nx.erdos_renyi_graph(1000,0.01,directed=True)
nx.draw_networkx(G, pos=nx.spring_layout(G), with_labels=False, node_size=10)
G.number_of_edges()

In [ ]:
import numpy as np

gmat = nx.to_numpy_matrix(G)

outmat = np.zeros(gmat.shape)

for i in range(gmat.shape[0]):
    for j in range(i+1,gmat.shape[1]):        
        outmat[i,j] = np.dot(np.asarray(gmat[i,:]).reshape(-1), np.asarray(gmat[j,:]).reshape(-1))
        
np.count_nonzero(outmat)

In [ ]:
%%timeit
for i in range(gmat.shape[0]):
    for j in range(i+1,gmat.shape[1]):        
        outmat[i,j] = np.dot(np.asarray(gmat[i,:]).reshape(-1), np.asarray(gmat[j,:]).reshape(-1))

That's a serial implementation. Let's parallelize it with `joblib`.

In [ ]:
def inner_loop(i):
    partial_out = np.zeros(gmat.shape[1])
    for j in range(i+1,gmat.shape[1]):    
        partial_out[j] = np.dot(np.asarray(gmat[i,:]).reshape(-1), np.asarray(gmat[j,:]).reshape(-1))    
    return partial_out
    
from joblib import Parallel, delayed
partials = Parallel(n_jobs=4)(delayed(inner_loop)(i) for i in range(gmat.shape[0]))

outmat = np.array(partials)
np.count_nonzero(outmat)

and time it

In [ ]:
%%timeit
partials = Parallel(n_jobs=4)(delayed(inner_loop)(i) for i in range(gmat.shape[0]))

OK, this is where we stopped in Lecture 2.

### Numba 

Numba is an open source JIT compiler that translates a subset of Python and NumPy code into fast machine code.  Let's look at the outcome of compiling the inner loop of our serial implementation. This should reduce the number of calls to the interpret to be O(rows) rather than O(rows*columns).

Read the 5-minute guide to Numba (https://numba.readthedocs.io/en/stable/user/5minguide.html)
  then do the following:
* import the numba jit
* add numba symbols and decorators to compile inner loop.

In [ ]:
# TODO

outmat = np.zeros(gmat.shape)

# TODO
def comp_inner_loop(i):
    partial_out = np.zeros(gmat.shape[1])
    for j in range(i+1,gmat.shape[1]):    
        partial_out[j] = np.dot(np.asarray(gmat[i,:]).reshape(-1), np.asarray(gmat[j,:]).reshape(-1))  
    return partial_out
    
for i in range(gmat.shape[0]):
    outmat[i,:] = comp_inner_loop(i)

np.count_nonzero(outmat)

In [ ]:
%%timeit
for i in range(gmat.shape[0]):
    outmat[i,:] = comp_inner_loop(i)

#### Numba and parallel for loops

OK, that is great. Let's try to compile the whole thing and add parralellism to the outer loop. You will need to check out explicit parallelization at https://numba.readthedocs.io/en/stable/user/parallel.html.

In [ ]:
# TODO

# TODO
def comp_inner_loop(i):
    partial_out = np.zeros(gmat.shape[1])
    for j in range(i+1,gmat.shape[1]):    
        partial_out[j] = np.dot(np.asarray(gmat[i,:]).reshape(-1), np.asarray(gmat[j,:]).reshape(-1))  
   # print(partial_out)
    return partial_out

# TODO
def comp_outerloop(outmat):

    for i in TODO range TODO (gmat.shape[0]):
        outmat[i,:] = comp_inner_loop(i)


outmat = np.zeros(gmat.shape)
comp_outerloop(outmat)
np.count_nonzero(outmat)

In [ ]:
%%timeit
comp_outerloop(outmat)

OK that was pretty awesome. >3x speedup over compiled code.

### Performance

Manually grab all the timings from the notebook and drop them in this plot.

In [ ]:
import pandas as pd

# get values from code
d = { 'loops': [3130], 'joblib': [2560], 'inner compiled': [233], 'parallel': [87] }
df = pd.DataFrame (data=d)
df.head()
df.plot.bar()

### Conclusions:

  * JIT Compilation is a powerful tool
  * JIT can only speed up the code that is compiled 
  * Numba has parallelization primitives for embarrassingly parallel code (only)
 
This is a pretty clear reminder that parallelism is only one of many speedup tools. It turns out to be useful here after we are running on compiled code, but in the big picture, compiling code to get access to ILP is equally important.

Numba gives access to vectorized code, but it has been hard to isolate. I'm working on it for another example.